In [1]:
import pandas as pd
from tqdm import tqdm

from uuid import uuid4

from langchain_core.documents import Document
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [2]:
# initializing data base for quadrant
client = QdrantClient(
    path='../db/qdrant_db',
)

/tmp/ipykernel_146953/5000106.py:2: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <books-rec-project> contains 67582 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client = QdrantClient(


In [3]:
# setting information about our data base
client.create_collection(
    collection_name='books-rec-project3',
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

True

In [4]:
# initializing model for embeddings
model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/o13g/bootcamp/ds-phase-2/rec-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('../data/genres/final.csv')

In [6]:
df.head()

,author,title,annotation,img_url,book_url,genre,clean_annotation,clean_title,clean_genre,clean_concat_content,main_genre
0,Miroshnik Mariya,Обещание Малфою,"Серый дым клубами поднимался все выше, постепе...",https://rust.litnet.com/uploads/covers/120/161...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",серый дым клуб подниматься высоко постепенно р...,обещание малфою,фанфик фанфики по фильмам,"Жанр: Фанфик, Фанфики по фильмам\nАннотация: С...",Фанфик
1,Антонина Аркадьева,Любовь На Эгейском Море _aŞk Sana Benzer,"Лето, теплое море и любовь простых красивых ду...",https://rust.litnet.com/uploads/covers/120/159...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",лето теплый море любовь простой красивый душа ...,любовь на эгейском море,фанфик фанфики по фильмам,"Жанр: Фанфик, Фанфики по фильмам\nАннотация: Л...",Фанфик
2,Этранжера,"Всё началось с поцелуя. (по сериалу ""Любовь на...","Омер и Дефне, знающий себе цену мужчина и неоп...",https://rust.litnet.com/uploads/covers/120/158...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",омер дефне знать цена мужчина неопытный девушк...,всё началось с поцелуя по сериалу любовь напрокат,фанфик фанфики по фильмам,"Жанр: Фанфик, Фанфики по фильмам\nАннотация: О...",Фанфик
3,Люба,Белла,Попаданка в тело Изабеллы Свон....,https://rust.litnet.com/uploads/covers/120/158...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",попаданка тело изабелла свон,белла,фанфик фанфики по фильмам,"Жанр: Фанфик, Фанфики по фильмам\nАннотация: П...",Фанфик
4,Cool blue lady,Мальчик голубых кровей. Испытание дружбой,Драко Малфой опускает голову и руки его трясут...,https://rust.litnet.com/uploads/covers/120/154...,https://litnet.com/ru/book/obeshchanie-malfoyu...,"Фанфик, Фанфики по фильмам",драко малфа опускать голова рука трястись подн...,мальчик голубых кровей испытание дружбой,фанфик фанфики по фильмам,"Жанр: Фанфик, Фанфики по фильмам\nАннотация: Д...",Фанфик


In [7]:
df['clean_concat_content'][1]

'Жанр: Фанфик, Фанфики по фильмам\nАннотация: Лето, теплое море и любовь простых красивых душой и телом людей. Любовь во всех её проявлениях, но не сиропно-розовая, а сдобренная колоритным турецким перчиком!...\nНазвание: Любовь На Эгейском Море _aŞk Sana Benzer'

In [8]:
documents = []
id_mapping = {}

for _, row in df.iterrows():
    content = row['clean_concat_content']
    
    metadata = {
        'author': row.get('author', ''),
        'title': row.get('title', ''),
        'main_genre': row.get('main_genre', ''),
        'annotation': row.get('annotation', ''),
        'img_url': row.get('img_url', ''),
        'book_url': row.get('book_url', '')
    }
    
    documents.append(Document(page_content=content, metadata=metadata))

uuids = [str(uuid4()) for _ in range(len(documents))]
print(f'Создано {len(documents)} документов с UUID')

Создано 67553 документов с UUID


In [9]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name='books-rec-project3',
    embedding=hf
)

In [10]:
with tqdm(total=len(documents), desc='Добавление документов в Qdrant') as pbar:
    for doc, doc_id in zip(documents, uuids):
        vector_store.add_documents(documents=[doc], ids=[doc_id])
        pbar.update(1)
        
    print(f'✅ {len(documents)} документов добавлено в Qdrant!')

Добавление документов в Qdrant:   0%|          | 0/67553 [00:00<?, ?it/s]

Добавление документов в Qdrant:  30%|██▉       | 20000/67553 [1:03:44<2:28:55,  5.32it/s]/home/o13g/bootcamp/ds-phase-2/rec-project/.venv/lib/python3.12/site-packages/langchain_qdrant/qdrant.py:444: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 20001 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  self.client.upsert(
Добавление документов в Qdrant: 100%|██████████| 67553/67553 [3:36:55<00:00,  5.19it/s]  

✅ 67553 документов добавлено в Qdrant!


In [11]:
query = 'Про мальчика и демона'
results = vector_store.similarity_search(
    query,
    k=3
)

In [12]:
results

[Document(metadata={'author': 'Ник Трейси', 'title': 'За демоном', 'main_genre': 'Молодежная проза', 'annotation': 'История о том, как обычные мальчишки пошли глубоко в дремучий лес, чтобы прикончить демона. ...', 'img_url': 'https://rust.litnet.com/uploads/covers/120/1540727763_43.jpg', 'book_url': 'https://litnet.com/ru/book/uslysh-menya-b108304', '_id': 'e353b7bc-8fd3-4175-abc5-4541dd050bfb', '_collection_name': 'books-rec-project3'}, page_content='Жанр: Молодежная проза, Молодежная мистика\nАннотация: История о том, как обычные мальчишки пошли глубоко в дремучий лес, чтобы прикончить демона. ...\nНазвание: За демоном'),
 Document(metadata={'author': 'RoMan', 'title': 'Влюблённый демон', 'main_genre': 'Молодежная проза', 'annotation': 'Демону запрещено любить человека, а если такое случается, сам дьявол будет пытать непослушного демона.\nГлавный герой, демон перекрёстка, сам того не замечая, влюбил...', 'img_url': 'https://rust.litnet.com/uploads/covers/120/1711516351_84.jpg', 'book

In [1]:
client.close()

NameError: name 'client' is not defined